# Рекомендация тарифов

**Описание проекта**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

**Цель проекта**

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Описание данных**

`/datasets/users_behavior.csv` 

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- 'сalls' — количество звонков;
- 'minutes' — суммарная длительность звонков в минутах;
- 'messages' — количество sms-сообщений;
- 'mb_used' — израсходованный интернет-трафик в Мб;
- 'is_ultra' — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [28]:
# импорт библиотек

import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

## Откройте и изучите файл

In [3]:
try:
    data = pd.read_csv('datasets/users_behavior.csv')
except FileNotFoundError as e:
    print(repr(e))
    data = pd.read_csv('/datasets/users_behavior.csv')

FileNotFoundError(2, 'No such file or directory')


In [4]:
# получаю общую сводку
data.info()

# проверю на корректный вывод
display(data.sample(5))

# проверяю имена колонок на пробелы
display(data.columns)

# проверю на явные дубликаты
print(f'Количество явных дубликатов: {data.duplicated().sum()}')

# описание
data.describe() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
230,36.0,267.58,13.0,12737.08,0
1903,134.0,940.77,56.0,2921.57,1
2084,41.0,296.80,70.0,17280.85,0
952,87.0,518.10,17.0,13957.77,0
133,109.0,704.30,79.0,10603.12,0


Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultra'], dtype='object')

Количество явных дубликатов: 0


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [25]:
print('Количество классов в выборке: \n')
print(data.is_ultra.value_counts())

print('\n ////////////////////////// \n')

print('В процентном соотношении:')
data.is_ultra.value_counts().div(
    data.is_ultra.value_counts().sum()
).mul(100).round(2)

Количество классов в выборке: 

0    2229
1     985
Name: is_ultra, dtype: int64

 ////////////////////////// 

В процентном соотношении:


0    69.35
1    30.65
Name: is_ultra, dtype: float64

In [26]:
data.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


### Предварительные замечания:

Замечаний по качеству данных нет. Данные соответствуют документации и не требуют предобработки.

Дисбаланс классов требует дополнительных метрик для оценки точности модели.  
Присутствует сильная корреляция между столбцами 'calls' и 'minutes'.

## Разбейте данные на выборки

Для исследования разобъю ихсодные данные на признаки и целевой признак где:

- признаки `features` это колонки 'calls', 'minutes', 'messages', 'mb_used';
- целевой признак `target` это колонка 'is_ultra'.

Полученные данные  разобъю на выборки в соотношении 3:1:1, где:
- 60% данных - обучающая выборка;
- 20% данных - валидационная выборка;
- 20% данных - тестовая выборка.

In [29]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

In [30]:
# воспользуюсь методом train_test_split чтобы отделить 40% данных, число random_state получено генератором случайных чисел
train_features, valid_features, train_target, valid_target = train_test_split(
    features,
    target,
    test_size=0.4,
    random_state=98470453
)

In [31]:
# воспользуюсь тем же методом для разделения валидационной и тестовой выборок
test_features, valid_features, test_target, valid_target = train_test_split(
    valid_features,
    valid_target,
    test_size=0.5,
    random_state=98470453
)

## Исследуйте модели

Для исследования воспользуюсь моделями Дерева Решений, Случайного Леса и Логистической Регрессии.  
Воспользуюсь методом `GridSearchCV` библиотеки `sklearn` для подбора гиперпараметров.

In [32]:
import warnings
warnings.filterwarnings('ignore')

### Дерево решений

Подготовлю модель.

In [33]:
dtc = DecisionTreeClassifier(random_state=98470453)

params_tree = [{
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2, 6),
    'min_samples_split': range(2, 6),
    'min_samples_leaf': range(2, 6),
    'min_weight_fraction_leaf': [0.0, 0.1, 0.2, 0.3]
}]

model_tree = GridSearchCV(dtc,
                      param_grid=params_tree,
                      scoring='accuracy',
                      cv=3)

model_tree.fit(train_features, train_target)
model_tree.best_params_

{'criterion': 'entropy',
 'max_depth': 5,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0}

Проверю модель на валидационной выборке.

In [43]:
valid_predictions_dtc = model_tree.predict(valid_features)

display(f'Accuracy: {accuracy_score(valid_target, valid_predictions_dtc)}')
        
display(f'Precision: {precision_score(valid_target, valid_predictions_dtc)}')
        
f'Recall: {recall_score(valid_target, valid_predictions_dtc)}'

'Accuracy: 0.807153965785381'

'Precision: 0.7807017543859649'

'Recall: 0.4734042553191489'

### Случайный лес

Подготовлю модель.

In [44]:
rfc = RandomForestClassifier(random_state=98470453)

params_forest = [{
    'criterion': ['gini', 'entropy'],
    'max_depth': range(3, 7),
    'n_estimators': range(2, 53, 5)
}]

model_forest = GridSearchCV(rfc,
                      param_grid=params_forest,
                      scoring='accuracy',
                      cv=3)

model_forest.fit(train_features, train_target)
model_forest.best_params_

{'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 27}

Проверю модель на валидационной выборке.

In [46]:
valid_predictions_rfc = model_forest.predict(valid_features)

display(f'Accuracy: {accuracy_score(valid_target, valid_predictions_rfc)}')
        
display(f'Precision: {precision_score(valid_target, valid_predictions_rfc)}')
        
f'Recall: {recall_score(valid_target, valid_predictions_rfc)}'

'Accuracy: 0.8133748055987559'

'Precision: 0.8269230769230769'

'Recall: 0.4574468085106383'

### Логистическая регрессия

Подготовлю модель.

In [47]:
lr = LogisticRegression(random_state=98470453)

params_regression = [{
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'max_iter': range(1, 101),
}]

model_regression = GridSearchCV(lr,
                      param_grid=params_regression,
                      scoring='accuracy')

model_regression.fit(train_features, train_target)
model_regression.best_params_

{'max_iter': 71, 'solver': 'lbfgs'}

Проверю модель на валидационной выборке.

In [48]:
valid_predictions_lr = model_regression.predict(valid_features)

display(f'Accuracy: {accuracy_score(valid_target, valid_predictions_lr)}')
        
display(f'Precision: {precision_score(valid_target, valid_predictions_lr)}')
        
f'Recall: {recall_score(valid_target, valid_predictions_lr)}'

'Accuracy: 0.7527216174183515'

'Precision: 0.7959183673469388'

'Recall: 0.2074468085106383'

### Вывод

По итогам подготовки и исследования моделей получены следующие результаты по метрике `accuracy`:
- точность ответов 0.8133 - модель Случайного Леса;
- точность ответов 0.8071 - модель Дерева Решений;
- точность ответов 0.7527 - модель Логистической Регрессии.

Всем тремя моделями на валидационной выборке достинут необходимый порог точности - 0.75.  
Для определения наиболее точной модели все три будут проверены на тестовой выборке.

Модели демонстрируют сравнимые с `accuracy` показатели метрики `precision`, то есть соотношение положительных результатов (1 предсказано как 1) к сумме положительных результатов и ложных положительных результатов (1 предсказано как 0).

Модели демотрируют низкие показатели (0.21 - 0.47) метрики `recall`, то есть соотношение положительных результатов (1 предсказано как 1) к сумме положительных результатов и ложных негативных результатов (0 предсказано как 1).

Низкие показатели метрики `recall` вызваны дисбалансом классов.

## Проверьте модель на тестовой выборке

### Дерево решений

In [53]:
%%time
test_predictions_dtc = model_tree.predict(test_features)
display(f'Accuracy: {accuracy_score(test_target, test_predictions_dtc)}')

'Accuracy: 0.8087091757387247'

CPU times: user 7.91 ms, sys: 38 µs, total: 7.95 ms
Wall time: 6.01 ms


In [54]:
display(f'Precision: {precision_score(test_target, test_predictions_dtc)}')
        
f'Recall: {recall_score(test_target, test_predictions_dtc)}'

'Precision: 0.8088235294117647'

'Recall: 0.5314009661835749'

### Случайный лес

In [55]:
%%time
test_predictions_rfc = model_forest.predict(test_features)
display(f'Accuracy: {accuracy_score(test_target, test_predictions_rfc)}')

'Accuracy: 0.8102643856920684'

CPU times: user 12.9 ms, sys: 7.95 ms, total: 20.8 ms
Wall time: 18.3 ms


In [58]:
display(f'Precision: {precision_score(test_target, test_predictions_rfc)}')
 
f'Recall: {recall_score(test_target, test_predictions_rfc)}'

'Precision: 0.8346456692913385'

'Recall: 0.5120772946859904'

### Логистическая регрессия

In [68]:
%%time
test_predictions_lr = model_regression.predict(test_features)
display(f'Accuracy: {accuracy_score(test_target, test_predictions_lr)}')

'Accuracy: 0.7387247278382582'

CPU times: user 8.58 ms, sys: 6 µs, total: 8.59 ms
Wall time: 6.4 ms


In [65]:
display(f'Precision: {precision_score(test_target, test_predictions_lr)}')
 
f'Recall: {recall_score(test_target, test_predictions_lr)}'

'Precision: 0.8421052631578947'

'Recall: 0.2318840579710145'

### Вывод

По итогам проверки моделей на тестовой выборке получены следующие результаты по метрике `accuracy`:

- точность ответов 0.8102 - модель Случайного Леса;
- точность ответов 0.8087 - модель Дерева Решений;
- точность ответов 0.7387 - модель Логистической Регрессии.

При проверке на тестовой выборке точность модели Логистической Регрессиина тестовой выборке не достигла 0.75.  

Модели демонстрируют сравнимые с `accuracy` показатели метрики `precision`, то есть соотношение положительных результатов (1 предсказано как 1) к сумме положительных результатов и ложных положительных результатов (1 предсказано как 0).

Модели демотрируют низкие показатели (0.23 - 0.53) метрики `recall`, то есть соотношение положительных результатов (1 предсказано как 1) к сумме положительных результатов и ложных негативных результатов (0 предсказано как 1).

Низкие показатели метрики `recall` вызваны дисбалансом классов.

Точность моделей Случайного Леса и Дерева Решений достаточна для выполнения поставленной задачи. При этом выполнение модели Дерева Решений занимает примерно в два раза меньше времени.

## (бонус) Проверьте модели на адекватность

Для проверки на адекватность использую DummyClassifier. Проверяю по метрике `accuracy`, как данной в задании.

In [83]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(random_state=98470453)

params_dummy = [{
    'strategy': ['most_frequent', 'prior', 'stratified', 'uniform', 'constant'],
    'constant': [0, 1]
}]

model_dummy = GridSearchCV(dummy,
                      param_grid=params_dummy,
                      scoring='accuracy')

model_dummy.fit(train_features, train_target)
model_dummy.best_params_

{'constant': 0, 'strategy': 'most_frequent'}

In [84]:
valid_predictions_dummy = model_dummy.predict(valid_features)

display(f'Accuracy: {accuracy_score(valid_target, valid_predictions_dummy)}')
        
display(f'Precision: {precision_score(valid_target, valid_predictions_dummy)}')
        
f'Recall: {recall_score(valid_target, valid_predictions_dummy)}'

'Accuracy: 0.7076205287713841'

'Precision: 0.0'

'Recall: 0.0'

In [85]:
test_predictions_dummy = model_dummy.predict(test_features)

display(f'Accuracy: {accuracy_score(test_target, test_predictions_dummy)}')

display(f'Precision: {precision_score(test_target, test_predictions_dummy)}')
 
f'Recall: {recall_score(test_target, test_predictions_dummy)}'

'Accuracy: 0.6780715396578538'

'Precision: 0.0'

'Recall: 0.0'

Точность моделей Случайного Леса и Дерева Решений выше точности фиктивной модели - следовательно полученные модели Случайного Леса и Дерева Решений адекватны.